In [45]:
import numpy
from PIL import Image
from IPython.display import Image as dispImage

In [46]:
#Generate initial image
data = numpy.zeros((8, 8, 3), dtype=numpy.uint8)

white = [255, 255, 255]
grey = [128, 128, 128]
black = [0,0,0]

boardRows = 8
boardColumns = 8

for i in range(boardRows):
  for j in range(boardColumns):
    if (i == 3 and j == 3) or (i == 4 and j == 4):
      data[i, j] = white
    elif (i == 3 and j == 4) or (i == 4 and j == 3):
      data[i, j] = black
    else:
      data[i, j] = grey


image = Image.fromarray(data)
image.show()
image = image.save("prueba.jpg")
dispImage("prueba.jpg", width=100, height=100)

In [47]:
#Path to the file 15x10_w.txt
with open('/content/drive/MyDrive/Colab Notebooks/Hackathon/Board_To_Image/Data/Training Set/15x10_w.txt') as f:
    lines = f.readlines()
    f.close()

In [102]:
auxParams = lines[0].split("\n")
params = auxParams[0].split(" ")
print(params)
#0 5 4 ==> 
#First   0 == black - 1 == white 
#Second  parameter column index (from 0 to 7) 
#Third   parameter row    index (from 0 to 7) 

['0', '5', '4']


In [103]:
#Print first movement
#FROM THIS POINTS, THE FIRST PARAMETER "i" IS THE ROW AND THE SECOND "j" THE COLUMN
if params[0] == "0":
  color = black
else:
  color = white
i = int(params[2])
j = int(params[1])
data[i,j] = color
print(i,j)  

4 5


In [ ]:
#Print all movements (WITHOUT FLIP TODO)
"""
for line in lines:
  auxParams = line.split("\n")
  params = auxParams[0].split(" ")
  if params[0] == "0":
    color = black
  else:
    color = white
  i = int(params[2]) #rows
  j = int(params[1]) #colummns
  data[i,j] = color
  print(data)  
"""

In [104]:
#Save image with movement
image = Image.fromarray(data)
image.show()
image = image.save("prueba.jpg")
dispImage("prueba.jpg", width=100, height=100)

In [132]:
#Detect the minimun or maximum spot in the same column
#Only returns the mininum or maximum spot
def detectSpotsByColumns(row, column, data):
  spotColor = black
  minColorColumn = 99
  maxColorColumn = -1

  newAffectedSpotCol = []

  for j in range(boardColumns):
    if j != column:
      if j < column and minColorColumn > j and data[row][j][0] == spotColor[0] and data[row][j][1] == spotColor[1] and data[row][j][2] == spotColor[2]:
        minColorColumn = j
      if j > column and maxColorColumn < j and data[row][j][0] == spotColor[0] and data[row][j][1] == spotColor[1] and data[row][j][2] == spotColor[2]:
        maxColorColumn = j    

  if maxColorColumn != -1:
    newAffectedSpotCol.append([row,maxColorColumn])
  if minColorColumn != 99:
    newAffectedSpotCol.append([row,minColorColumn])
  return newAffectedSpotCol


In [136]:
spotAffectedByColumn = detectSpotsByColumns(4, 5, data) #4,5,data
print("Spot on Columns: ",spotAffectedByColumn)

Spot on Columns:  [[4, 3]]


In [137]:
#Detect the minimun or maximum spot in the same column
#Only returns the mininum or maximum spot
def detectSpotsByRow(row, column, data):
  minColorRow = 99
  maxColorRow = -1
  newAffectedSpotRow = []

  for i in range(boardRows):
    if i != row:
      if i < row and minColorRow > i and data[i][column][0] == spotColor[0] and data[i][column][1] == spotColor[1] and data[i][column][2] == spotColor[2]:
        minColorRow = i
      if i > row and maxColorRow < i and data[i][column][0] == spotColor[0] and data[i][column][1] == spotColor[1] and data[i][column][2] == spotColor[2]:
        maxColorRow = i  

  if maxColorRow != -1:
    newAffectedSpotRow.append([maxColorRow,column])
    print(maxColorRow)
  if minColorRow != 99:
    newAffectedSpotRow.append([minColorRow,column])
    print(minColorRow)

  print(newAffectedSpotRow)

spotAffectedByRow = detectSpotsByRow(4,5,data)
print("Spot on Rows: ",spotAffectedByRow)

[]
Spot on Rows:  None


In [139]:
#Detect the minimun or maximum spot in the same diagonal
#Returns a dictionary, with this possible keys:
#             "lu" (Left  Up Diagonal)
#             "ld" (Left  Down Diagonal)
#             "ru" (Right Up Diagonal)
#             "rd" (Right Down Diagonal)

def detectSpotsByDiagonals(row, column, data):

  lu = [99,99] # Left Up 
  ld = [99,-1] # Left Down
  ru = [-1,99] # Right Up
  rd = [-1,-1] # Right Down

  finalData = dict()

  for i in range(boardRows):
    for j in range(boardColumns):
      if abs(i-j) == abs(row - column) or i+j == row + column:
        if i != row and j != column: 
          if i < row and j < column:
            if lu[0] > i and lu[1] > j and data[i][j][0] == spotColor[0] and data[i][j][1] == spotColor[1] and data[i][j][2] == spotColor[2]:
              lu = [i,j]
          elif i < row and j > column:
            if ld[0] > i and ld[1] < j and data[i][j][0] == spotColor[0] and data[i][j][1] == spotColor[1] and data[i][j][2] == spotColor[2]:
              ld = [i,j]
          elif i > row and j < column:
            if ru[0] < i and ru[1] > j and data[i][j][0] == spotColor[0] and data[i][j][1] == spotColor[1] and data[i][j][2] == spotColor[2]:
              ru = [i,j]
          elif i > row and j > column:
            if rd[0] < i and rd[1] < j and data[i][j][0] == spotColor[0] and data[i][j][1] == spotColor[1] and data[i][j][2] == spotColor[2]:
              rd = [i,j]

  if lu[0] != 99:
    finalData["lu"] = [lu]
  if ld[0] != 99:
    finalData["ld"] = [ld]
  if ru[0] != -1:
    finalData["ru"] = [ru]
  if rd[0] != -1:
    finalData["rd"] = [rd]

  return finalData

nameDiagDiags = detectSpotsByDiagonals(4, 5, data) 
print("Spots on Diagonals: ",nameDiagDiags)   

Spots on Diagonals:  {'lu': [[3, 4]]}
